In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from keras_preprocessing.image import ImageDataGenerator

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
trainlabels = pd.read_csv("/kaggle/input/cifar-10/trainLabels.csv")


In [ ]:
display(trainlabels)

In [ ]:
!pip install py7zr

In [ ]:
import py7zr
temp_path = '/kaggle/temp/'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
train_file_path = '/kaggle/input/cifar-10/train.7z'
archive = py7zr.SevenZipFile(train_file_path, mode='r')
archive.extractall(path=temp_path)
archive.close()


In [ ]:
temp_path = '/kaggle/test/'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
test_file_path = '/kaggle/input/cifar-10/test.7z'
archive_test = py7zr.SevenZipFile(test_file_path, mode='r')
archive_test.extractall(path=temp_path)
archive.close()

In [ ]:
os.listdir(temp_path)

In [ ]:
train_data = os.path.join(temp_path,'train')

In [ ]:
sns.countplot(trainlabels['label'])

In [ ]:
def append_ext(fn):
    return "/kaggle/working/train/"+str(fn)+'.png'

In [ ]:
trainlabels['path'] = trainlabels['id']
trainlabels['path'] = trainlabels['path'].apply(append_ext)
datagen=ImageDataGenerator(rotation_range=15,width_shift_range=4,height_shift_range=4, validation_split=0.2, rescale=1./255.)
bs=64

In [ ]:
trainlabels.head()

In [ ]:
testlabels = pd.read_csv("/kaggle/input/cifar-10/Labels.csv")

In [ ]:
train_generator=datagen.flow_from_dataframe(
                        dataframe=trainlabels,
                        x_col="path",
                        y_col="label",
                        batch_size=bs,
                        seed=42,
                        subset = 'training',
                        shuffle=True,
                        class_mode="categorical",
                        target_size =  (32,32))

valid_generator=datagen.flow_from_dataframe(
                        dataframe=testlabels,
                        x_col="path",
                        y_col="label",
                        batch_size=bs,
                        seed=42,
                        subset = 'validation',
                        shuffle=True,
                        class_mode="categorical",
                        target_size =  (32,32))